In [1]:
from rdflib import Graph, Literal, RDF, URIRef
# rdflib knows about quite a few popular namespaces, like W3C ontologies, schema.org etc.
from rdflib.namespace import FOAF , XSD
from rdflib.namespace import NamespaceManager
from rdflib import BNode
import pandas as pd
import numpy as np
from uuid import uuid4

In [2]:
def function(q):
    df = pd.read_csv('qa.csv')
    idx = df.index[df['question'].str.strip() == q].tolist()
    maxac = 0.0
    maxa = ''
    for i in idx:
        ac = df.iloc[i]['accuracy']
        if ac > maxac:
            maxac = ac
            maxa = df.iloc[i]['answer']
    return maxa

In [3]:
function('What is Artificial Intelligence?')

"The term Artificial Intelligence (AI) is derived from the English word AI which is derived from the Latin word intellectusI feel that the definition of Artificial Intelligence is very flexible and can be found in every context; it's just a question of usageExamples:For the purposes of this question, we will use the following example"

In [104]:
def queryQAstatement(namespace):
    #ns1:d3ace751-eef8-4ec0-8a33-0af17a99b8fa ns1:outcomes
    kg = Graph()
    kg.parse("KGLayer/commonkg.n3")
    q = """
        PREFIX ns1: <"""+namespace+"""/>
        SELECT ?t ?o
        WHERE {?t ns1:outcomes ?o .}"""
    #print(q)
    qr = kg.query(q)
    kg.close()
    qa = []
    if len(qr)==0:
        return 0
    else:
        for r in qr:
            task = r['t'].split('/')[-1]
            ctx = findContext(task, namespace)
            #print(ctx)
            if ctx!= 0:
                reward = findPolicy(ctx.split('/')[-1],namespace)
                out = str(r.asdict()['o'].toPython())
                q = out.split('\n')[0]
                if (len(out.split('\n'))>1):
                    a = ''.join(out.split('\n')[1:])
                    a = a.split('.')[:-2]
                    a = ''.join(a)
                else:
                    a = ''
                qa.append([q, a, float(reward)])
        df = pd.DataFrame(qa, columns=['question','answer','accuracy'])
        df.to_csv('qa.csv')
    #for si in services:
     #   if si in service_list:
      #      #print(si, services)
        return 1
    #kg.serialize(destination='KGLayer/commonkg'+".n3")
    #kg.close()

In [105]:
queryQAstatement('http://aimicroservice.derby.ac.uk')

                            question  \
0                         basketball   
1                         basketball   
2  What is Artificial Intelligence?    

                                              answer  accuracy  
0  What's it like being a baseball fan? Why do th...       0.6  
1                                                          0.9  
2  The term Artificial Intelligence (AI) is deriv...       0.8  


1

In [39]:
def findContext(taskid, namespace):
    #ns1:28910ed6-d4b2-4e6c-8401-0e13f552c2e5 a ns1:task ;
    #ns1:context ns1:d966b50b-a5f6-4c9f-938c-01548e8ae599 ;
    #ns1:desire_output [ ns1:iodatatype ns1:txt ] ;
    #ns1:input [ ns1:iodatatype ns1:jpeg ] .
    cg = Graph()
    cg.parse("KGLayer/contextkg.n3")
    q = """
        PREFIX ns1: <"""+namespace+"""/>
        SELECT ?t ?ctx
        WHERE {ns1:"""+taskid+""" ns1:context ?ctx .}"""
    #print(q)
    qr = cg.query(q)
    cg.close()
    if len(qr)==0:
        return 0
    else:
        for r in qr:
            c = r["ctx"]
            #print (c)
        return c    

In [96]:
def findPolicy(context,namespace):
    #ns1:8c43ad1e-dcb0-4e2f-9f4f-6cd89d60e956 ns1:policy_state "0.9"@en ;
    #ns1:c5dbe1c6-f224-4ac2-97ee-cd6b7c0c0f04 ns1:reward "0.8"@en;
    pg = Graph()
    pg.parse("KGLayer/policy.n3")
    q = """
        PREFIX ns1: <"""+namespace+"""/>
        SELECT ?r
        WHERE {{ns1:"""+context+""" ns1:policy_state ?r .} UNION {ns1:"""+context+""" ns1:reward ?r .} }"""
    #print(q)
    qr = pg.query(q)
    pg.close()
    if len(qr)==0:
        return 0
    else:
        for r in qr:
            r = r.asdict()['r'].toPython()
            #print (c)
        return r    